## Google Drive initializaton 

In [ ]:
import os
colab = True #If you are running colab
if colab:
    from google.colab import drive
    drive.mount('/content/drive', force_remount= True)
    try:
        path = '/content/drive/MyDrive/Operating System Project'
        os.chdir(path)
    except:
        path = '/content/drive/MyDrive/FIU/Classes/Operative Systems/Final Project' # insert your path to the shared folder
    os.chdir(path)
!ls

## Library Imports

In [ ]:
from IPython.display import clear_output
import sklearn
import numpy as np
import pandas as pd
import tensorflow as tf
import random, string
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 18, 'font.family': 'STIXGeneral', 'mathtext.fontset':'cm'})

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,LearningCurveDisplay, learning_curve
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
save = False # if you want to save figures

## `Cache` object
Creates a simple cache object for simulation purposes. Considers a cache with slots available for caching files. One file per slot, we don't consider allocation properties.

It requires three parameters:

-`memory_size`: Maximum amount of elements on the cache

-`set_function`: A function in charge of performing the cache replacement. It has to recieve:
- `Cache`: A `Cache` object
- `timestamp`: Time when the key was requested
- `key`:       Name of element requested by the cache
- `value` (optional): Additional value. For instnace, probability of being 
requested next time

-`key list`: List with names of the cached elements

We implemented LRU (Least Recently Used) MRU (Most Recently Used) cache policies.




In [ ]:
class Cache:
    def __init__(self, memory_size, set_function, key_list):
        
        self.size = memory_size
        self.cachevalue = {} # Value associated to the file (for example, probability)
        self.timestamps = {} # Last time updated/requested
        self.set_function = set_function
        self.timelastuse = {key: 0 for key in key_list } # Last time updated/requested for each
        self.usenumber = {key:0 for key in key_list } #stores how many times the key has been requested

        self.cache_hit = 0
        self.cache_miss = 0
        self.cache_hit_performance = np.empty(shape=(0, ))
        self.time = 0
        self.views = {key: 0 for key in key_list} # Number of views for each movie

    def Get(self, timestamp, key, value = None ): # Queries an element from the cache memory, returns the file if it is in cache, else it returns -1
        self.usenumber[key] += 1
        self.timelastuse[key] = timestamp-self.timelastuse[key]
        if key in self.cachevalue:
            self.timestamps[key] = timestamp
            if value is not None:
              self.cachevalue[key]['value'] = value
            return self.cachevalue[key]
        return -1
    
    def Set(self, timestamp, key, value=None):
        return self.set_function(self, timestamp, key, value)

    def Simulate(self, inqury, value = None, view_increment = 1):
        content = self.Get(self.time, inqury, value)
        if content == -1:

            self.cache_miss += 1
            self.Set(self.time, inqury, value)
            
        else:
            self.cache_hit += 1
            
        

        # Sort cache based on 'views' attribute
        self.cache_hit_performance = np.hstack((self.cache_hit_performance, self.cache_hit))
        self.time += 1
 

    
"""With cache hit, view will be increased and the the cache will be sorted based on it. 
It means most popular content will be at the top"""
   

'''Replacement functions. Each function has to recieve:
- A Cache object
- A timestamp: time when the key was requested
- A key:       Name of element requested by the cache
- A value (optional): Additional value. For instnace, probability of being requested next time

It makes the replacement, replaces and old key with the new key
'''

def Set_LRU(cache_obj,timestamp, key, value=None): # Replacement policy if a change has to be made, in this case it is LRU. However, it can be changed
    if len(cache_obj.cachevalue) >= cache_obj.size:

        # find the oldest entry in the cache
        oldest_key = min(cache_obj.timestamps, key=cache_obj.timestamps.get)

        # take out the oldest key
        cache_obj.cachevalue.pop(oldest_key)
        cache_obj.timestamps.pop(oldest_key)

    # update values, regardless if it was a cache hit or not
    cache_obj.cachevalue[key] = value
    cache_obj.timestamps[key] = timestamp


def Set_MRU(cache_obj,timestamp, key, value=None): # Replacement policy if a change has to be made, in this case it is MRU. However, it can be changed
    if len(cache_obj.cachevalue) >= cache_obj.size:
        
        # find the oldest entry in the cache
        newest_key = max(cache_obj.timestamps, key=cache_obj.timestamps.get)

        # take out the oldest key
        cache_obj.cachevalue.pop(newest_key)
        cache_obj.timestamps.pop(newest_key)

    # update values, regardless if it was a cache hit or not
    cache_obj.cachevalue[key] = {'value': value, 'views': 0}
    cache_obj.timestamps[key] = timestamp

##Simulation example

Creates 100 random words and simulate a Cache memory with `memory_size` size and simulates `n_inquiries` requests drawn at uniformly random.

In [ ]:
words_list = []
for _ in range(100):
    words_list.append(''.join(random.choices(string.ascii_uppercase + string.digits, k=10)))

In [ ]:
# simulation
n_inquiries, memory_size = 10000, 10

# cache_memory, time, cache_hit, cache_miss, cache_hit_performance = Initialize(n_inquiries, memory_size,words_list, Set_MRU)
cache_memory = Cache(memory_size= memory_size, set_function =Set_LRU, key_list =words_list)

cache_inquries = random.choices(words_list, k = n_inquiries)

for inqury in cache_inquries:
    cache_memory.Simulate(inqury)


cache_hit, cache_miss, cache_hit_performance, time = cache_memory.cache_hit, cache_memory.cache_miss, cache_memory.cache_hit_performance, cache_memory.time

print(f'Cache hits {cache_hit}\n Cache misses {cache_miss}')

#plots
f, (ax1,ax2) = plt.subplots(1, 2, figsize=(15,4))
ax1.plot(100*cache_hit_performance/n_inquiries)
ax1.set_title('Cache saturation through execution')
ax1.set_xlabel('Inquiry')
ax1.set_ylabel('Cache hit [%]')
ax2.set_title('Relative cache')
ax2.plot(100*cache_hit_performance/ range(1,n_inquiries+1))
ax2.set_xlabel('Inquiry')
ax2.set_ylabel('Current cache hit [%]')

# print(len(cache_memory.usenumber), len(cache_memory.timelastuse))
""" Note that this simulation doesn't involve the movie dataset or the popularity feature, as it is a separate demonstration of the cache functionality."""

## Reading dataset
Reads an preprocessed IMdB dataset `Toy dataset.csv`, which is sorted by the most reviewed movies and shows the `n_movies` most reviewed ones. 

In [ ]:
dataset_df = pd.read_csv('Toy dataset.csv')
dataset_df = dataset_df.drop('endYear', axis =1)
dataset_df = dataset_df.replace('\\N',0)
n_movies = 500 # the most reviewed movies
dataset_df= dataset_df[:n_movies]
fig = plt.figure(figsize=(8,5))
plt.plot(dataset_df['numVotes'])
plt.title('Movie distribution by popularity')
plt.xlabel(r'$n^{th}$ most popular movie')
plt.ylabel('Number of votes')
dataset_df.head(10)
if save:
    fig.savefig('movie distribution.eps', format ='eps',bbox_inches='tight', transparent = True)

## Feature extractors

Auxiliar functions that obtain features to be used in the ML models. They inclde columns such that `'averageRating','runtimeMinutes', 'isAdult', 'startYear'`

In [ ]:
def One_hot(values, length= n_movies):

    encoding = [0.0]*length
    for value in values:
        encoding[value]= 1.0
    return encoding

   
def get_popularity(cache_obj, movie, days=30):
    current_time = cache_obj.time
    start_time = max(current_time - days, 0)
    views = [cache_obj.views[movie] for t in range(start_time, current_time)]
    popularity = sum(views) / days
    return popularity



def Obtain_features(cache_obj, df, current_inquiry, 
                    cache_inquiries):
    # current inquiry is an index
    time = cache_obj.time
    num_negative_per_positive = 1
    last_inquiries = cache_inquiries[time-num_of_past_inquiries:time]

    feature_list = [Obtain_individual_features(cache_obj, df, current_inquiry, 
                    last_inquiries)]

    label_list = [1.0]
    index_list = np.array(list(converter_dict.keys()))
    del_list = np.delete(index_list, current_inquiry)
    neg_class = np.random.choice(del_list,size=num_negative_per_positive, replace=False)

    for non_movie in neg_class:

        
        feature_list.append(Obtain_individual_features(cache_obj, df, non_movie, 
                    last_inquiries))
        label_list += [-1.0]

    return feature_list, label_list

def Obtain_individual_features(cache_obj, df, current_inquiry, 
                    last_inquiries):
    columns = ['averageRating','runtimeMinutes', 'isAdult', 'startYear' ]
    num_of_past_inquiries = 5
    popularity = get_popularity(cache_obj, current_inquiry)
    lst = One_hot([current_inquiry]) + [cache_obj.timelastuse[current_inquiry],
            cache_obj.usenumber[current_inquiry], popularity] +  df.loc[current_inquiry,columns].to_list()

    lst +=  One_hot(last_inquiries)
    return lst


## Simulating cache accesses
Simulates cache accesses which drawing samples at random from the distribution defined by their probability in order to have a dataset to train a ML model.

In [ ]:
n_inquiries, memory_size, num_of_past_inquiries = 20000, 25, 5

cache_memory = Cache(memory_size,Set_LRU, range(n_movies))

probability_distribution = np.array(dataset_df['numVotes']/(dataset_df['numVotes'].sum()))

np.random.seed(12345)
cache_inquiries = np.random.choice(range(n_movies), n_inquiries, p = probability_distribution )
  

converter_dict = {index : movie for movie, index in zip(dataset_df['originalTitle'].to_list(),range(n_movies) )}


X_dataset, y_dataset = [], []

for movie in cache_inquiries:
    cache_memory.Simulate(movie)
    if cache_memory.time>num_of_past_inquiries-1:

        # print(last_inquries)
        X, y = Obtain_features(cache_memory,dataset_df, movie,
                            cache_inquiries)
        X_dataset += X
        y_dataset += y
    if cache_memory.time %100==0:
        clear_output(wait=True)
        print(f'Query number {cache_memory.time}')

X_dataset, y_dataset = np.array(X_dataset, dtype=np.float32), np.array(y_dataset, dtype=np.float32)

print(f'Dataset with {X_dataset.shape[0]} samples and {X_dataset.shape[1]} features each.')

## Training a ML model

In [ ]:
from sklearn.model_selection import train_test_split
proportion = 0.8
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset,train_size = proportion)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## Plot results simulated cache access

In [ ]:
cache_hit, cache_miss, cache_hit_performance, time = cache_memory.cache_hit, cache_memory.cache_miss, cache_memory.cache_hit_performance, cache_memory.time
print(f'Cache hits: {cache_hit} \nCache misses: {cache_miss}')

plt.figure(figsize=(7,4))
plt.plot(100*cache_hit_performance/n_inquiries)
plt.title('Cache hit saturation')
plt.ylabel('Relative hit [%]')
plt.xlabel('Query number')
fig = plt.figure(figsize=(7,4))
plt.plot(100*cache_hit_performance/ range(1,n_inquiries+1))
plt.title('Cache hit classic LRU')
plt.ylabel('Cache hit rate [%]')
plt.xlabel('Query number')
print(X_dataset.shape, y_dataset.shape)
print(f'Memory size {memory_size}')
if save:
    fig.savefig(f'OS_Only_LRU_Cache_Hit_with_{n_inquiries}.eps', format ='eps',bbox_inches='tight', transparent = True)

## Machine Learning models

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def get_model(kind='knn'):
    if kind == 'svm':
        model = make_pipeline(StandardScaler(), SVC(C=10.0**(2), gamma='auto',verbose =True, probability =True))
    elif kind == 'mlp':
        model = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(128,64,),verbose =False))
    elif kind == 'forest':
        model = make_pipeline(StandardScaler(), RandomForestClassifier(max_features= 'auto', max_depth = 70))
    elif kind == 'knn':
        model = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=10)) #number 10 is giving better accuracy in test and train set
    
    return model
kind = 'mlp'
model = get_model(kind)
proportion = 0.8
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset,train_size = proportion)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) 

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_predicted = model.predict(X_test)
test_acc, train_acc = accuracy_score(y_predicted, y_test), accuracy_score(y_train, model.predict(X_train))

print(f'training acc {train_acc}. test acc {test_acc}')

## Learning rates

In [ ]:
learning_curv = True
if learning_curv:
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train)
    display = LearningCurveDisplay(train_sizes=train_sizes,
                                train_scores=train_scores, test_scores=test_scores)
    fig,ax =plt.subplots(figsize=(8,5))
    display.plot(ax, score_type ='both')
    ax.set_title(f'Learning Curves {kind.upper()} model')
    ax.set_xlabel('Number of samples/ Number of cache queries')
    ax.set_ylabel('Accuracy')
    if save:
        fig.savefig(f'{kind.upper()}_Learning_Curve_Accuracy_in_{n_inquiries}.svg', format ='svg',bbox_inches='tight', transparent = True)

## New cache replacement policies

In [ ]:
def Set_ML_MRU(model, df = dataset_df,cache_inquiries = cache_inquiries, num_of_past_inquiries =num_of_past_inquiries): 

    def set_function(cache_obj,timestamp, key, value =None):
        last_inquiries =  cache_inquiries[timestamp-num_of_past_inquiries:timestamp]
        if len(cache_obj.cachevalue) >= cache_obj.size:
            for movie in cache_obj.cachevalue.keys():
                features = Obtain_individual_features(cache_obj, df, movie, 
                    last_inquiries)
                features = np.array([features])
                cache_obj.cachevalue[movie] = model.predict_proba(features)[0,0]

            most_likely_key = max(cache_obj.cachevalue, key=cache_obj.cachevalue.get)


            # take out the oldest key
            cache_obj.cachevalue.pop(most_likely_key)
            cache_obj.timestamps.pop(most_likely_key)

        # update values, regardless if it was a cache hit or not
        cache_obj.cachevalue[key] = {'value': value, 'views': 0}
        cache_obj.timestamps[key] = timestamp
    return set_function

def Set_ML_LRU(model, df = dataset_df,cache_inquiries = cache_inquiries, num_of_past_inquiries =num_of_past_inquiries): 

    def set_function(cache_obj,timestamp, key, value =None):
        last_inquiries = last_inquiries = cache_inquiries[timestamp-num_of_past_inquiries:timestamp]
        if len(cache_obj.cachevalue) >= cache_obj.size:
            for movie in list(cache_obj.cachevalue.keys()):                
                features = Obtain_individual_features(cache_obj, df, movie, 
                    last_inquiries )
                features = np.array([features])
                cache_obj.cachevalue[movie] = model.predict_proba(features)[0,1]
                # print(model.predict_proba(features))

            
            least_likely_key = min(cache_obj.cachevalue, key=cache_obj.cachevalue.get)
            # print(f'k {most_likely_key}')

            # take out the oldest key
            cache_obj.cachevalue.pop(least_likely_key)
            cache_obj.timestamps.pop(least_likely_key)

        # update values, regardless if it was a cache hit or not
        cache_obj.cachevalue[key] = {'value': value, 'views': 0}
        cache_obj.timestamps[key] = timestamp
    return set_function

## Cache replacement aided by ML simulation

In [ ]:
cache_memory = Cache(memory_size, Set_ML_LRU(model), range(n_movies))


for movie in cache_inquiries:
    cache_memory.Simulate(movie)
    if cache_memory.time%100==0:
        clear_output(wait=True)
        print(f'Query number {cache_memory.time}')


cache_hit, cache_miss, cache_hit_performance, time = cache_memory.cache_hit, cache_memory.cache_miss, cache_memory.cache_hit_performance, cache_memory.time


## Plot results of aided cache by ML

In [ ]:
cache_hit, cache_miss, cache_hit_performance, time = cache_memory.cache_hit, cache_memory.cache_miss, cache_memory.cache_hit_performance, cache_memory.time
print(f'Cache hits: {cache_hit} \nCache misses: {cache_miss}')
plt.figure(figsize=(7,4))
plt.plot(100*cache_hit_performance/n_inquiries)
plt.title('Cache hit saturation')
plt.ylabel('Relative hit [%]')
plt.xlabel('Query number')
fig = plt.figure(figsize=(7,4))
plt.plot(100*cache_hit_performance/ range(1,n_inquiries+1))
plt.title(f'Cache hit LRU aided by {kind.upper()}')
plt.ylabel('Cache hit rate [%]')
plt.xlabel('Query number')
if save:
    fig.savefig(f'{kind.upper()}_LRU_Cache_Hit_{n_inquiries}.eps', format ='eps',bbox_inches='tight', transparent = True)
print(X_dataset.shape, y_dataset.shape)
print(f'Memory size {memory_size}')